<a href="https://colab.research.google.com/github/AbdalazizOth/Tradtional_dict/blob/main/Ubab_Zakhir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[UbabZakhir_dict](https://github.com/OpenITI/0650AH/tree/master/data/0650IbnMuhammadRadiDinSaghani/0650IbnMuhammadRadiDinSaghani.CubabZakhir)**

In [6]:
import re                                # RegEx module for regular expressions
import pandas as pd                      # Python library for data manipulation
from openpyxl import load_workbook       # Python library to read/write Excel files
from openpyxl.styles import Alignment

# File paths
input_file = "/content/0650IbnMuhammadRadiDinSaghani.CubabZakhir.Shamela0000761-ara1"
output_file_txt = "UbabZakhir.txt"
output_file_xlsx = "UbabZakhir.xlsx"

# Regex patterns
page_pattern = re.compile(r"PageV\d{2}P\d+")                             # Page number pattern like PageV01P66
root_pattern = re.compile( r"^(?:### \|\s*(.{3,})|#\s*([ء-ي]{3})$)") # Root pattern (starting with ### | or #)

# Initialize variables
data = []
page = None          # Current active page
next_page = None     # Temporarily holds the next page marker (for future root)
root = None
text_lines = []

# Process the file
with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        line = line.rstrip()
        if re.match(r"^# فر.",line):
           continue

 # Detect page marker (may appear within text)
        if match := page_pattern.search(line):
            next_page = match.group(0)           # store for the next root
            line = page_pattern.sub("", line)    # remove marker but keep text
            line = line.strip()                  # trim spaces after removal


         # Detect new root
        if match := root_pattern.match(line):
            # Save previous root (if one exists)
            if root and text_lines:
                text = " ".join(text_lines).strip()
                data.append((page, root, text))
                text_lines = []


            # Assign correct page for this new root
            # If a new page marker was found → use it
            # Otherwise → keep using the previous page (same page for multiple roots)
            if next_page is not None:
                page = next_page
                next_page = None


            # Extract new root and first line of text
            root = match.group(1) or match.group(2)
            root = re.sub(r"\bms\d{3}\b", "", root)
            root = re.sub(r"[:()]","",root)
            root = root.strip()


        # Collect text lines under current root
        elif root and line.strip():
            cleaned = line.lstrip("#~: ").strip()
            cleaned = re.sub(r"\bms\d{3}\b|«\d+»|\|\||\.\.\.|«", "", cleaned)
            if cleaned:
                text_lines.append(cleaned)

# Save the last root at end of file
if root and text_lines:
    text = " ".join(text_lines).strip()
    data.append((page, root, text))

# 1️⃣ Save text file
with open(output_file_txt, "w", encoding="utf-8") as out:
    for page, root, text in data:
        out.write(f"Page: {page}\nRoot: {root}\nText: {text}\n\n")

# 2️⃣ Save Excel file
df = pd.DataFrame(data, columns=["Page", "Root", "Text"])
df.to_excel(output_file_xlsx, index=False, engine="openpyxl")

# 3️⃣ Format Excel (RTL + wrap text)
wb = load_workbook(output_file_xlsx)
ws = wb.active

ws.sheet_view.rightToLeft = True

align = Alignment(wrap_text=True, horizontal="right", vertical="top")
for row in ws.iter_rows():
    for cell in row:
        cell.alignment = align

# Adjust column width (fixed reasonable size)
ws.column_dimensions["A"].width = 15   # Page
ws.column_dimensions["B"].width = 20   # Root
ws.column_dimensions["C"].width = 80   # Text

wb.save(output_file_xlsx)

print(f"✅ Done! {len(data)} roots extracted.")  # to determine the number of items in data list
print(f"📄 Text file: {output_file_txt}")
print(f"📊 Excel file: {output_file_xlsx}")

✅ Done! 1669 roots extracted.
📄 Text file: UbabZakhir.txt
📊 Excel file: UbabZakhir.xlsx
